In [1]:
!pip install yfinance plotly streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 48.8 MB/s eta 0:00:00


In [2]:
import yfinance as yf
import pandas as pd
import plotly.graph_objs as go
from datetime import datetime, timedelta


In [4]:
import yfinance as yf
import pandas as pd

# Fetch last 3 months of data
ticker = 'AAPL'
data = yf.download(ticker, period='3mo', interval='1d')

# Display first few rows
print(data.head())

# Save data to CSV file
csv_filename = f"{ticker}_3months_data.csv"
data.to_csv(csv_filename, index=True)

print(f"✅ Data saved successfully as '{csv_filename}'")


/tmp/ipython-input-960741203.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period='3mo', interval='1d')
[*********************100%***********************]  1 of 1 completed

Price            Close        High         Low        Open     Volume
Ticker            AAPL        AAPL        AAPL        AAPL       AAPL
Date                                                                 
2025-08-08  229.090012  230.738135  219.001455  220.579666  113854000
2025-08-11  227.179993  229.559998  224.759995  227.919998   61806100
2025-08-12  229.649994  230.800003  227.070007  228.009995   55626200
2025-08-13  233.330002  235.000000  230.429993  231.070007   69878500
2025-08-14  232.779999  235.119995  230.850006  234.059998   51916300
✅ Data saved successfully as 'AAPL_3months_data.csv'


In [5]:
# Moving Averages
data['SMA_20'] = data['Close'].rolling(window=20).mean()
data['SMA_50'] = data['Close'].rolling(window=50).mean()

# RSI Calculation
def compute_rsi(data, window=14):
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

data['RSI'] = compute_rsi(data)
data.tail()


Price,Close,High,Low,Open,Volume,SMA_20,SMA_50,RSI
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,,,
Date,,,,,,,,
2025-11-03,269.049988,270.850006,266.250000,270.420013,50194600,259.127499,249.074199,77.708349
2025-11-04,270.040009,271.489990,267.619995,268.329987,49274800,259.805499,249.931799,77.366506
2025-11-05,270.140015,271.700012,266.929993,268.609985,43683100,260.409499,250.748400,81.487688
2025-11-06,269.769989,273.399994,267.890015,267.890015,51204000,261.195999,251.533999,77.693265
2025-11-07,268.470001,272.290009,266.769989,269.799988,48203600,262.355999,252.252199,63.596704


In [6]:
fig = go.Figure()

# Add candlestick chart
fig.add_trace(go.Candlestick(
    x=data.index,
    open=data['Open'],
    high=data['High'],
    low=data['Low'],
    close=data['Close'],
    name='Candlestick'
))

# Add moving averages
fig.add_trace(go.Scatter(x=data.index, y=data['SMA_20'], line=dict(width=2), name='SMA 20'))
fig.add_trace(go.Scatter(x=data.index, y=data['SMA_50'], line=dict(width=2), name='SMA 50'))

fig.update_layout(title=f"{ticker} Stock Price with Indicators", xaxis_title="Date", yaxis_title="Price (USD)")
fig.show()


In [7]:
%%writefile stock_dashboard.py
import yfinance as yf
import pandas as pd
import plotly.graph_objs as go
import streamlit as st
from datetime import datetime, timedelta

st.set_page_config(page_title="Stock Market Dashboard", layout="wide")

st.title("📈 Stock Market Data Dashboard")

# Sidebar for user input
ticker = st.sidebar.text_input("Enter Stock Symbol (e.g., AAPL, TSLA, MSFT):", "AAPL")
period = st.sidebar.selectbox("Select Period", ["1mo", "3mo", "6mo", "1y", "2y"], index=1)
interval = st.sidebar.selectbox("Select Interval", ["1d", "1h", "15m"], index=0)

# Fetch data
data = yf.download(ticker, period=period, interval=interval)

# Calculate Indicators
data['SMA_20'] = data['Close'].rolling(window=20).mean()
data['SMA_50'] = data['Close'].rolling(window=50).mean()

# RSI Function
def compute_rsi(data, window=14):
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

data['RSI'] = compute_rsi(data)

# Plot
fig = go.Figure()
fig.add_trace(go.Candlestick(x=data.index, open=data['Open'], high=data['High'], low=data['Low'], close=data['Close'], name='Candlestick'))
fig.add_trace(go.Scatter(x=data.index, y=data['SMA_20'], line=dict(width=2), name='SMA 20'))
fig.add_trace(go.Scatter(x=data.index, y=data['SMA_50'], line=dict(width=2), name='SMA 50'))

fig.update_layout(title=f"{ticker} Price Chart", xaxis_title="Date", yaxis_title="Price (USD)")

st.plotly_chart(fig, use_container_width=True)

# RSI Chart
st.subheader("Relative Strength Index (RSI)")
st.line_chart(data['RSI'])


Writing stock_dashboard.py


In [8]:
!streamlit run stock_dashboard.py





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.142.246.179:8501

  Stopping...
